In [ ]:
import scipy.stats as sts
import numpy as np

In [1]:
def distribution(rho: float, population_size: int):
    return sts.norm(rho, ((1 + rho**2) / population_size) ** 0.5)


def generate_population(rho: float, population_size: int, number_samples: int):
    p_value_estimator = (
        lambda statistic: distribution(0, population_size).sf(statistic) * 2
    )
    dist = sts.multivariate_normal([0, 0], [[1, rho], [rho, 1]])
    for _ in range(number_samples):
        samples = dist.rvs(size=population_size)
        yield p_value_estimator(abs((samples[:, 0] @ samples[:, 1]) / population_size))

In [2]:
n_rejection = int


def without_group_correction(p_values_spread: list[float], alpha: float) -> n_rejection:
    return sum(p < alpha for p in p_values_spread)


def bonferroni(p_values_spread: list[float], alpha: float) -> n_rejection:
    return sum(p < alpha / len(p_values_spread) for p in p_values_spread)


def benjamini_hochberg(p_values_spread: list[float], alpha: float) -> n_rejection:
    sorted_inds = np.argsort(p_values_spread)
    comparison_vec = p_values_spread[sorted_inds] <= np.ones(
        len(p_values_spread)
    ) * alpha / len(p_values_spread)
    reject_cut_off_index = (
        np.argmax(np.nonzero(comparison_vec)[0]) + 1 if comparison_vec.sum() > 0 else 0
    )

    return reject_cut_off_index

In [ ]:
n_trials = 100

In [ ]:
sum(
    bonferroni(
        list(generate_population(rho=0, population_size=100, number_samples=500)), 0.05
    )
    > 0
    for _ in range(n_trials)
)